In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install -q pydrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [6]:
!pip install transformers

     |████████████████████████████████| 2.3MB 14.0MB/s 
     |████████████████████████████████| 3.3MB 44.1MB/s 
     |████████████████████████████████| 901kB 43.7MB/s 


In [7]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 15.1MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 1.2MB 20.0MB/s 
     |████████████████████████████████| 1.8MB 47.5MB/s 
     |████████████████████████████████| 225kB 42.0MB/s 
     |████████████████████████████████| 122kB 45.6MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 8.2MB 47.4MB/s 
     |████████████████████████████████| 174kB 47.4MB/s 
     |████████████████████████████████| 133kB 54.6MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 245kB 45.1MB/s 
     |████████████████████████████████| 112kB 51.7MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 4.2MB 33.0MB/s 
     |████████████████████████████████| 112kB 54.9MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 122kB 52.6MB/s

In [14]:
project_path='/content/drive/My Drive/NLP/'

In [17]:
import numpy as np
import pandas as pd

def create_dataset():
  fake = pd.read_csv(project_path+'fake_and_real_news/Fake.csv')
  real = pd.read_csv(project_path+'fake_and_real_news/True.csv')
  fake['label'] = np.negative(np.ones((len(fake['text']))))
  real['label'] = np.ones((len(real['text'])))
  df = pd.DataFrame({'text':[], 'label':[]}, columns=['text', 'label']);
  combined = pd.concat([fake, real, df])
  return combined

dataset = create_dataset()

#dataset = pd.read_csv(project_path+'fake_or_real_news.csv')
print(dataset.shape)

texts=[]
texts=dataset['text']#####################################
label=dataset['label']

from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))


#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

training_size=int(0.8*dataset.shape[0])
print(dataset.shape[0],training_size)
data_train=dataset[:training_size]['text']
y_train_val=y[:training_size]
data_test=dataset[training_size:]['text']
y_test=y[training_size:]


(44898, 5)
44898 35918


In [18]:
sentences = data_train
print(sentences)
labels = y_train_val[:,0]
#labels = y_train
print(labels)

0        Donald Trump just couldn t wish all Americans ...
1        House Intelligence Committee Chairman Devin Nu...
2        On Friday, it was revealed that former Milwauk...
3        On Christmas day, Donald Trump announced that ...
4        Pope Francis used his annual Christmas Day mes...
                               ...                        
12432    SYDNEY (Reuters) - Australian Prime Minister M...
12433    BEIJING (Reuters) - China s Foreign Ministry o...
12434    HONG KONG (Reuters) - Three judges in Hong Kon...
12435    KABUL (Reuters) - One Afghan and three Turkish...
12436    MOSCOW (Reuters) - A Russian defense ministry ...
Name: text, Length: 35918, dtype: object
[0 0 0 ... 1 1 1]


In [19]:
print('Max sentence length: ', max([len(sen.split(' ')) for sen in sentences]))

Max sentence length:  8436


In [20]:
print(sentences[:1])

0    Donald Trump just couldn t wish all Americans ...
Name: text, dtype: object


In [21]:
TRAIN_SPLIT = 0.8

nb_train_samples = int(TRAIN_SPLIT * sentences.shape[0])

x_train = np.array(sentences[:nb_train_samples])
y_train = np.array(labels[:nb_train_samples])
x_val = np.array(sentences[nb_train_samples:])
y_val = np.array(labels[nb_train_samples:])


In [22]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [23]:
sentences_test = np.array(data_test)
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test)

WARSAW (Reuters) - Poland s new government led by Prime Minister Mateusz Morawiecki won a vote of confidence in parliament just before midnight on Tuesday, voting records showed, opening the way for the Cabinet to start functioning. Morawiecki, 49, was named prime minister last week in a government reshuffle, replacing Beata Szydlo as the ruling Law and Justice party gears up for elections over the next three years. He said Warsaw s economic policy - based on generous public spending and a growing focus on building domestic capital - should not change, but his government would aim to improve Poland s external relations. 
[1 1 1 ... 1 1 1]


In [24]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [25]:
print('Training sample: ' + str(len(train_df['text'])))
print('Validation sample: ' + str(len(val_df['text'])))
print('Testing sample: ' + str(len(test_df['text'])))

Training sample: 28734
Validation sample: 7184
Testing sample: 8980


In [27]:
model_type = "roberta"
model_name = "roberta-base"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": "/content/drive/My Drive/NLP/Transformer/output/"+model_type,
    "best_model_dir": "/content/drive/My Drive/NLP/Transformer/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News FR_1 RoBerta",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [28]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = True


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=train_args)

# Train the model
model.train_model(train_df,eval_df = val_df)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:800: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)


(320,
 {'auprc': [1.0, 1.0, 0.9999999999999999, 1.0, 1.0],
  'auroc': [nan, nan, nan, nan, nan],
  'eval_loss': [1.113296312568462,
   0.018149966246291862,
   0.024829828513650207,
   0.022584566382889068,
   0.020863131230050055],
  'fn': [7184, 21, 24, 23, 19],
  'fp': [0, 0, 0, 0, 0],
  'global_step': [64, 128, 192, 256, 320],
  'mcc': [0.0, 0.0, 0.0, 0.0, 0.0],
  'tn': [0, 0, 0, 0, 0],
  'tp': [0, 7163, 7160, 7161, 7165],
  'train_loss': [0.4454224407672882,
   0.02657604217529297,
   0.0004944148822687566,
   0.00037512570270337164,
   0.00020491777104325593]})

In [29]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:800: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)


Training loss,0.0003
lr,2e-05
global_step,320
_runtime,198
_timestamp,1620967569
_step,10
tp,7165
tn,0
fp,0
fn,19
mcc,0.0


Training loss,█▁▁▁▁▁
lr,▁▂▄▅▇█
global_step,▁▁▂▃▄▅▅▆▆▇█
_runtime,▁▂▃▄▄▅▅▆▇▇█
_timestamp,▁▂▃▄▄▅▅▆▇▇█
_step,▁▂▂▃▄▅▅▆▇▇█
tp,▁████
tn,▁▁▁▁▁
fp,▁▁▁▁▁
fn,█▁▁▁▁
mcc,▁▁▁▁▁


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:800: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [30]:
predictions, raw_outputs = model.predict(test_df['text'])

In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_df['labels'],predictions))

0.7978841870824054


In [32]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'wb') as f:
  pickle.dump((test_df['labels'],predictions),f)


FileNotFoundError: ignored

In [33]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'rb') as f:
  y_true, y_pred = pickle.load(f)

FileNotFoundError: ignored

In [34]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

NameError: ignored